[Hanane D](https://www.linkedin.com/in/hanane-d-algo-trader)

How to Evaluate a RAG Pipeline Using LlamaIndex and Giskard AI for a 10K Financial Report?


I used **LlamaParse** from LlamaIndex to parse Amazon 10K financial report, and used Giskard AI to evaluate the RAG pipeline.


With **Giskard AI**, you can generate a testset to/and evaluate your RAG app.

There are two main components in the RAG evaluation Toolkit:

**1-** **The testset generation** component uses **RAGET** (RAG Evaluation Toolkit) to automatically generate a dataset consisting of a list of question, reference answers, and reference contexts. The last one is retrieved from your knowledge base.

**2-**The **evaluation** component will use the previously generated test set to assess the correctness of your RAG application. The goal is to evaluate each part of your RAG system and identify areas that need improvement. You can use **RAGAS metrics (context recall, context precision, faithfulness, and answer relevancy)**

# Install Lib

In [ ]:
!pip install llama-index llama-index-core llama-parse openai llama_index.embeddings.huggingface -q
!pip install llama-index-llms-anthropic -q

# Specify API Keys

In [4]:
import nest_asyncio
nest_asyncio.apply()

from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
LLAMAPARSE_API_KEY = userdata.get('LLAMACLOUD_API_KEY')

# Loading financial report: Amazon 2023 10K

In [5]:
!wget "https://d18rn0p25nwr6d.cloudfront.net/CIK-0001018724/c7c14359-36fa-40c3-b3ca-5bf7f3fa0b96.pdf" -O amzn_2023_10k.pdf

--2024-08-25 12:53:06--  https://d18rn0p25nwr6d.cloudfront.net/CIK-0001018724/c7c14359-36fa-40c3-b3ca-5bf7f3fa0b96.pdf
Resolving d18rn0p25nwr6d.cloudfront.net (d18rn0p25nwr6d.cloudfront.net)... 99.84.178.124, 99.84.178.109, 99.84.178.193, ...
Connecting to d18rn0p25nwr6d.cloudfront.net (d18rn0p25nwr6d.cloudfront.net)|99.84.178.124|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 800598 (782K) [application/pdf]
Saving to: ‘amzn_2023_10k.pdf’

amzn_2023_10k.pdf   100%[===================>] 781.83K  --.-KB/s    in 0.01s   

2024-08-25 12:53:06 (63.1 MB/s) - ‘amzn_2023_10k.pdf’ saved [800598/800598]



# LlamaParse: Amazon 10K Financial report

In [6]:
from llama_parse import LlamaParse
import nest_asyncio;
nest_asyncio.apply()

pdf_name = "amzn_2023_10k.pdf"
# set up parser
parser = LlamaParse(api_key=LLAMAPARSE_API_KEY, result_type="markdown", gpt4o_mode = True)
documents = parser.load_data(pdf_name)

Started parsing the file under job_id 9b70c4ce-60bd-4415-ae20-64883043b3aa
.

In [7]:
from llama_index.core.node_parser import SentenceSplitter

######## SentenceSplitter ########
splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

######## Vector Index ########
from llama_index.core import VectorStoreIndex

embed_model = "local:BAAI/bge-small-en-v1.5" #https://huggingface.co/collections/BAAI/bge-66797a74476eb1f085c7446d
vector_index = VectorStoreIndex(nodes, embed_model = embed_model)

######## GPT-4o to Chat ########
from llama_index.llms.openai import OpenAI

llm_gpt4o = OpenAI(model="gpt-4o-mini", api_key = OPENAI_API_KEY)
query_engine_gpt4o = vector_index.as_query_engine(similarity_top_k=3, llm=llm_gpt4o)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
print(nodes[0].text)

## Store documents: with embeddings for later retrieval

In [ ]:
# vector_index.storage_context.persist(persist_dir=path)

## Chatting with the LLMs: GPT-4o-mini

In [64]:
query1 = "What is the net income on 2023?"
resp = query_engine_gpt4o.query(query1)
print("GPT-4o-mini:")
print(str(resp))

GPT-4o-mini:
The net income for 2023 is not explicitly provided in the context information. However, it can be inferred from the components of income before taxes and the provision for income taxes. The income (loss) before income taxes for 2023 is $37,557 million, and the provision for income taxes is $7,120 million. Therefore, the net income for 2023 can be calculated as follows:

Net Income = Income (loss) before income taxes - Provision for income taxes
Net Income = $37,557 million - $7,120 million = $30,437 million.

Thus, the net income for 2023 is approximately $30.4 billion.


# Giskard AI: 1- Generating testset

In [ ]:
!pip install "giskard[llm]" -q

## Generate a test set on the 10k report

In [12]:
import pandas as pd

In [13]:
from giskard.rag import KnowledgeBase, generate_testset, QATestset

knowledge_base_df = pd.DataFrame([node.text for node in nodes], columns=["text"])

In [14]:
# #Number of clusters
# import numpy as np
# round(2 + np.log(len(knowledge_base_df)))

7

In [15]:
# WORKS
import giskard
from giskard.llm.client.openai import OpenAIClient

import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

giskard.llm.set_llm_api("openai")
gpt4o_mini = OpenAIClient(model="gpt-4o-mini")
giskard.llm.set_default_client(gpt4o_mini)

knowledge_base = KnowledgeBase(knowledge_base_df, llm_client = giskard.llm.set_default_client(gpt4o_mini))

In [16]:
%%time
testset = generate_testset(knowledge_base,
                           num_questions=60,
                           agent_description="A chatbot answering questions about the Amazon 10K financial report of 2023")

INFO:giskard.rag:Finding topics in the knowledge base.
INFO:giskard.rag:Found 3 topics in the knowledge base.


Generating questions:   0%|          | 0/60 [00:00<?, ?it/s]

CPU times: user 12.2 s, sys: 158 ms, total: 12.4 s
Wall time: 1min 59s


In [ ]:
testset.to_pandas().head(5)

In [17]:
df_testset = testset.to_pandas()

## Different type of questions: 6

In [18]:
df_testset['question_type']=df_testset['metadata'].apply(lambda x: x['question_type'])

In [21]:
df_testset['question_type'].unique()

array(['simple', 'complex', 'distracting element', 'situational',
       'double', 'conversational'], dtype=object)

In [20]:
df_testset.groupby(['question_type'])['question'].count() #reamember:  num_questions=60, ==> 6 * 10

,question
question_type,
complex,10
conversational,10
distracting element,10
double,10
simple,10
situational,10


### Simple

In the simple question generation, Giskard AI asks the LLM to generate a pair of question and answer related the given context (chunk).

- By reading the prompt template, you'll understand clearly how it's built:
https://github.com/Giskard-AI/giskard/blob/main/giskard/rag/question_generators/simple_questions.py


In [28]:
df_simple = df_testset.query("question_type=='simple'")

#Take one example:
ref_context = df_simple['reference_context'].iloc[7]

#See how the other type of questions have been built by Giskard, based on the same chunk:
df_example = df_testset.query("reference_context==@ref_context")
for i in range(len(df_example)):
  print("\ni=",i)
  print("\nQUESTION_TYPE")
  print(df_example['question_type'].iloc[i])
  print("\nQUESTION")
  print(df_example['question'].iloc[i])

  print("\nreference_answer")
  print(df_example['reference_answer'].iloc[i])
  print("--"*50)


i= 0

QUESTION_TYPE
simple

QUESTION
What was the total lease cost for the year ended December 31, 2023?

reference_answer
$18,918 million
----------------------------------------------------------------------------------------------------

i= 1

QUESTION_TYPE
distracting element

QUESTION
Considering the tax-related items that need to be collected before the release of shares, what was the total lease cost recognized in 2023?

reference_answer
$18,918 million
----------------------------------------------------------------------------------------------------


In [30]:
#Take another example:
ref_context = df_simple['reference_context'].iloc[8]

#See how the other type of questions have been built by Giskard:
df_example = df_testset.query("reference_context==@ref_context")
for i in range(len(df_example)):
  print("\ni=",i)
  print("\nQUESTION_TYPE")
  print(df_example['question_type'].iloc[i])
  print("\nQUESTION")
  print(df_example['question'].iloc[i])

  print("\nreference_answer")
  print(df_example['reference_answer'].iloc[i])
  print("--"*50)


i= 0

QUESTION_TYPE
simple

QUESTION
What is the definition of fair value in financial instruments?

reference_answer
Fair value is defined as the price that would be received to sell an asset or paid to transfer a liability in an orderly transaction between market participants at the measurement date.
----------------------------------------------------------------------------------------------------

i= 1

QUESTION_TYPE
conversational

QUESTION
What does it mean?

reference_answer
Fair value is defined as the price that would be received to sell an asset or paid to transfer a liability in an orderly transaction between market participants at the measurement date.
----------------------------------------------------------------------------------------------------


### Complex

The complex question is generated by taking a simple question and asking the LLM to reformulate it into a more elaborate version, considering the context. However, the LLM is not asked to answer the reformulated question.

I've noticed that some 'simple' and 'complex' questions result in the same answer. I would prefer to see more elaborated answers for the complex questions. In the prompt associated to the "complex", only the question is reformulated based on the context.


In [42]:
df_complex = df_testset.query("question_type=='complex'")

#Take one example:
ref_context = df_complex['reference_context'].iloc[7]

#See how the other type of questions have been built by Giskard, based on the chunk:
df_example = df_testset.query("reference_context==@ref_context")
for i in range(len(df_example)):
  print("\ni=",i)
  print("\nQUESTION_TYPE")
  print(df_example['question_type'].iloc[i])
  print("\nQUESTION")
  print(df_example['question'].iloc[i])

  print("\nreference_answer")
  print(df_example['reference_answer'].iloc[i])
  print("--"*50)


i= 0

QUESTION_TYPE
complex

QUESTION
Could you identify the individual serving as the Executive Chair of the Board of Directors, including any relevant background information that highlights their experience or contributions to the company?

reference_answer
Jeffrey P. Bezos
----------------------------------------------------------------------------------------------------

i= 1

QUESTION_TYPE
situational

QUESTION
As a savvy investor analyzing Amazon's 2023 10K report for potential gains, I'm curious to know who currently holds the position of Executive Chair of the Board of Directors?

reference_answer
Jeffrey P. Bezos
----------------------------------------------------------------------------------------------------


### Distracting element

In [38]:
df_distracting = df_testset.query("question_type=='distracting element'")

#Take one example:
ref_context = df_distracting['reference_context'].iloc[7]

#See how the other type of questions have been built by Giskard, based on the chunk:
df_example = df_testset.query("reference_context==@ref_context")
for i in range(len(df_example)):
  print("\ni=",i)
  print("\nQUESTION_TYPE")
  print(df_example['question_type'].iloc[i])
  print("\nQUESTION")
  print(df_example['question'].iloc[i])

  print("\nreference_answer")
  print(df_example['reference_answer'].iloc[i])
  print("--"*50)


i= 0

QUESTION_TYPE
simple

QUESTION
What was the total lease cost for the year ended December 31, 2023?

reference_answer
$18,918 million
----------------------------------------------------------------------------------------------------

i= 1

QUESTION_TYPE
distracting element

QUESTION
Considering the tax-related items that need to be collected before the release of shares, what was the total lease cost recognized in 2023?

reference_answer
$18,918 million
----------------------------------------------------------------------------------------------------


### Situational

In [ ]:
df_situational = df_testset.query("question_type=='situational'")

#Take one example:
ref_context = df_situational['reference_context'].iloc[0]

#See how the other type of questions have been built by Giskard, based on the chunk:
df_example = df_testset.query("reference_context==@ref_context")
for i in range(len(df_example)):
  print("\ni=",i)
  print("\nQUESTION_TYPE")
  print(df_example['question_type'].iloc[i])
  print("\nQUESTION")
  print(df_example['question'].iloc[i])

  print("\nreference_answer")
  print(df_example['reference_answer'].iloc[i])
  print("--"*50)

### Double

In [46]:
df_double = df_testset.query("question_type=='double'")

#Take one example:
ref_context = df_double['reference_context'].iloc[5]

#See how the other type of questions have been built by Giskard, based on the chunk:
df_example = df_testset.query("reference_context==@ref_context")
for i in range(len(df_example)):
  print("\ni=",i)
  print("\nQUESTION_TYPE")
  print(df_example['question_type'].iloc[i])
  print("\nQUESTION")
  print(df_example['question'].iloc[i])

  print("\nreference_answer")
  print(df_example['reference_answer'].iloc[i])
  print("--"*50)


i= 0

QUESTION_TYPE
simple

QUESTION
What was the opinion of the independent registered public accounting firm on Amazon.com, Inc.'s financial statements as of December 31, 2023?

reference_answer
In our opinion, the consolidated financial statements present fairly, in all material respects, the financial position of the Company at December 31, 2023 and 2022, and the results of its operations and its cash flows for each of the three years in the period ended December 31, 2023, in conformity with U.S. generally accepted accounting principles.
----------------------------------------------------------------------------------------------------

i= 1

QUESTION_TYPE
double

QUESTION
What was the opinion on Amazon.com, Inc.'s consolidated financial statements and the effectiveness of its internal control over financial reporting as of December 31, 2023?

reference_answer
In our opinion, the consolidated financial statements present fairly the financial position of the Company at December 31

### Conversational

In [57]:
df_conversational = df_testset.query("question_type=='conversational'")

#Take one example:
ref_context = df_conversational['reference_context'].iloc[6]

#See how the other type of questions have been built by Giskard, based on the chunk:
df_example = df_testset.query("reference_context==@ref_context")
for i in range(len(df_example)):
  print("\ni=",i)
  print("\nQUESTION_TYPE")
  print(df_example['question_type'].iloc[i])
  print("\nQUESTION")
  print(df_example['question'].iloc[i])

  print("\nreference_answer")
  print(df_example['reference_answer'].iloc[i])
  print("--"*50)


i= 0

QUESTION_TYPE
simple

QUESTION
What is the definition of fair value in financial instruments?

reference_answer
Fair value is defined as the price that would be received to sell an asset or paid to transfer a liability in an orderly transaction between market participants at the measurement date.
----------------------------------------------------------------------------------------------------

i= 1

QUESTION_TYPE
conversational

QUESTION
What does it mean?

reference_answer
Fair value is defined as the price that would be received to sell an asset or paid to transfer a liability in an orderly transaction between market participants at the measurement date.
----------------------------------------------------------------------------------------------------


# Giskard AI: 2- Evaluation of the RAG pipeline

https://docs.giskard.ai/en/stable/open_source/testset_generation/rag_evaluation/index.html

In [ ]:
!pip install langchain_core -q
!pip install ragas -q

In [60]:
!pip install pyarrow -q

In [61]:
from giskard.rag import evaluate, RAGReport
from giskard.rag.metrics.ragas_metrics import ragas_context_recall, ragas_context_precision, ragas_faithfulness, ragas_answer_relevancy

In [ ]:
def answer_fn(question):
    answer = query_engine_gpt4o.query(question)
    return str(answer)

report = evaluate(answer_fn,
                testset=testset,
                knowledge_base=knowledge_base,
                metrics=[ragas_context_recall, ragas_context_precision, ragas_faithfulness, ragas_answer_relevancy])

In [63]:
display(report.to_html(embed=True))

/usr/local/lib/python3.10/dist-packages/IPython/core/display.py:724: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


# Key Takeaways

**1-** You can specify the LLM you want to use for test set generation (see the Notebook example where I used GPT-4o-mini). If no model is specified, the default LLM is GPT-4, so be mindful of the associated costs.

**2-** It's an interesting approach to use different types of questions. However, I also find it valuable to generate answers based on these question types to achieve more precise responses. For example, I've noticed that some 'simple' and 'complex' questions result in the same answer. I would prefer to see more elaborated answers for the complex questions. In the prompt associated to the "complex", only the question is reformulated based on the context.

**3-** Topics extracted from Amazon financial report are not relevant, I would prefer, for example, find different parts like "Sales", "Liquidity and Capital Resources", "Segments" topics...


- It's really interesting framework to consider!
